# Deduplicate DISPATCHLOAD

In [1]:
import os
import json
import importlib

import polars as pl
from tqdm import tqdm

import pyarrow
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.csv

# utils is our local utility module
# if we change utils.py, and re-run a normal 'import'
# python won't reload it by default. (Since it's already loaded.)
# So we force a reload
import utils
importlib.reload(utils)

<module 'utils' from '/home/matthew/Documents/TSE/AppliedEconometrics/repo/utils.py'>

## Constants

In [2]:
repo_data_dir = '/home/matthew/Documents/TSE/AppliedEconometrics/repo/data/'
laptop_data_dir = '/home/matthew/data/'

# result of the previous script
source_dir = os.path.join(laptop_data_dir, '01-D-parquet-pyarrow-dataset/DISPATCHLOAD')
temp_dir = os.path.join(laptop_data_dir, '03-A-parquet-pyarrow-dataset-duplicated-partitioned/DISPATCHLOAD')
dest_dir = os.path.join(laptop_data_dir, '03-B-parquet-pyarrow-dataset-deduplicated-partitioned/DISPATCHLOAD')

schema_path = os.path.join(repo_data_dir, 'schemas.json')

## Preparation

In [3]:
logger = utils.Logger(os.path.join(repo_data_dir, 'logs.txt'), flush=True)
logger.info("Initialising Logger")

In [4]:
utils.renice()

In [5]:
with open(schema_path, 'r') as f:
    schemas = json.load(f)

## Check size of each table

In [6]:
utils.create_dir(dest_dir)
utils.create_dir(temp_dir)

In [7]:
table = 'DISPATCHLOAD'
schema = {c: aemo_type_to_arrow_type(t['AEMO_type'], date_as_str=False) for (c,t) in schemas[table]['columns'].items()}
schema.update({
    "SCHEMA_VERSION": pa.int8(), 
    "TOP_TIMESTAMP": pa.string(),
})
dataset = ds.dataset(
    source=source_dir, 
    format="parquet",
    schema=pyarrow.schema(schema)
)

dest_partition_cols = ['DUID']

shutil.rmtree(temp_dir, ignore_errors=True)
ds.write_dataset(
    data=dataset, 
    base_dir=temp_dir, 
    format="parquet", 
    min_rows_per_group=512,
    partitioning=ds.partitioning(
        pa.schema({c:t for (c,t) in schema.items() if c in dest_partition_cols}),
        flavor="hive"
    ),
    existing_data_behavior="delete_matching"
)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (4040206031.py, line 21)